In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import BatchNormalization, Activation
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
df = pd.read_csv('./merged_file_F1_data.csv')

<ipython-input-6-f63b05b4eff5>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./merged_file_F1_data.csv')


In [ ]:
print(df['positionOrder'])

0         1
1         2
2         3
3         4
4         5
         ..
26075    16
26076    17
26077    18
26078    19
26079    20
Name: positionOrder, Length: 26080, dtype: int64


In [ ]:
columns_to_delete = ['round', 'name', 'date', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'resultId', 'number', 'position', 'positionText','points', 'laps','milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'time_x', 'time_y']

for column in columns_to_delete:
    del df[column]

columns = df.columns.tolist()
print(columns)


['raceId', 'year', 'circuitId', 'driverId', 'constructorId', 'grid', 'positionOrder']


In [ ]:
df.loc[df['positionOrder'] > 3, 'positionOrder'] = 0

In [ ]:
position = df['positionOrder']
print(position)

0        1
1        2
2        3
3        0
4        0
        ..
26075    0
26076    0
26077    0
26078    0
26079    0
Name: positionOrder, Length: 26080, dtype: int64


In [ ]:
data_types = df.dtypes

print(data_types)

df

raceId           int64
year             int64
circuitId        int64
driverId         int64
constructorId    int64
grid             int64
positionOrder    int64
dtype: object


,raceId,year,circuitId,driverId,constructorId,grid,positionOrder
0,1,2009,1,18,23,1,1
1,1,2009,1,22,23,2,2
2,1,2009,1,15,7,20,3
3,1,2009,1,10,7,19,0
4,1,2009,1,4,4,10,0
...,...,...,...,...,...,...,...
26075,1110,2023,13,817,213,19,0
26076,1110,2023,13,858,3,18,0
26077,1110,2023,13,807,210,0,0
26078,1110,2023,13,832,6,4,0


In [ ]:
features = ['raceId', 'year', 'circuitId', 'driverId', 'constructorId', 'grid']
target = ['positionOrder']

In [ ]:
x = df[features]
y = df['positionOrder']

In [ ]:
y_array = np.array(y)
y_reshaped = y_array.reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y_reshaped)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
x_onehot = encoder.fit_transform(x)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_onehot, y_onehot, test_size=0.3,random_state=109)

In [ ]:

model_01 = keras.Sequential([
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(4, activation="softmax")
])


In [ ]:
model_02 = keras.Sequential([
 layers.Dense(128, activation="relu"),
 layers.Dense(128, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(64, activation="relu"),
 layers.Dense(4, activation="softmax")
])

In [ ]:
model = models.Sequential()
model.add(layers.InputLayer(input_shape=(2343,)))
model.add(layers.Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))  # Optional dropout layer for regularization
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer="adam",
 loss="categorical_crossentropy",
 metrics=["accuracy"])

In [ ]:
model_01.compile(optimizer="adam",
 loss="categorical_crossentropy",
 metrics=["accuracy"])

In [ ]:
initial_learning_rate = 0.001
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)

In [ ]:
model_02.compile(optimizer="adam",
 loss="categorical_crossentropy",
 metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test))

Epoch 1/20
36/36 [==============================] - 4s 52ms/step - loss: 2.0473 - accuracy: 0.3266 - val_loss: 1.0001 - val_accuracy: 0.8767
Epoch 2/20
36/36 [==============================] - 1s 40ms/step - loss: 1.5173 - accuracy: 0.4391 - val_loss: 0.7901 - val_accuracy: 0.8767
Epoch 3/20
36/36 [==============================] - 2s 56ms/step - loss: 1.2115 - accuracy: 0.5563 - val_loss: 0.6498 - val_accuracy: 0.8767
Epoch 4/20
36/36 [==============================] - 2s 69ms/step - loss: 0.9709 - accuracy: 0.6680 - val_loss: 0.5492 - val_accuracy: 0.8767
Epoch 5/20
36/36 [==============================] - 1s 39ms/step - loss: 0.8118 - accuracy: 0.7492 - val_loss: 0.5071 - val_accuracy: 0.8767
Epoch 6/20
36/36 [==============================] - 1s 36ms/step - loss: 0.6984 - accuracy: 0.7985 - val_loss: 0.4901 - val_accuracy: 0.8767
Epoch 7/20
36/36 [==============================] - 1s 36ms/step - loss: 0.6189 - accuracy: 0.8295 - val_loss: 0.4787 - val_accuracy: 0.8767
Epoch 8/20
36

In [ ]:
history_01 = model_01.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test))

Epoch 1/20
36/36 [==============================] - 4s 30ms/step - loss: 0.8868 - accuracy: 0.8448 - val_loss: 0.4591 - val_accuracy: 0.8767
Epoch 2/20
36/36 [==============================] - 1s 22ms/step - loss: 0.4215 - accuracy: 0.8724 - val_loss: 0.3897 - val_accuracy: 0.8767
Epoch 3/20
36/36 [==============================] - 1s 21ms/step - loss: 0.3745 - accuracy: 0.8724 - val_loss: 0.3773 - val_accuracy: 0.8767
Epoch 4/20
36/36 [==============================] - 1s 21ms/step - loss: 0.3539 - accuracy: 0.8724 - val_loss: 0.3780 - val_accuracy: 0.8767
Epoch 5/20
36/36 [==============================] - 1s 20ms/step - loss: 0.3353 - accuracy: 0.8728 - val_loss: 0.3829 - val_accuracy: 0.8781
Epoch 6/20
36/36 [==============================] - 1s 21ms/step - loss: 0.3123 - accuracy: 0.8843 - val_loss: 0.4115 - val_accuracy: 0.8713
Epoch 7/20
36/36 [==============================] - 1s 23ms/step - loss: 0.2800 - accuracy: 0.8931 - val_loss: 0.4467 - val_accuracy: 0.8650
Epoch 8/20
36

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
history_02 = model_02.fit(X_train, y_train, epochs=20, batch_size=512, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20
36/36 [==============================] - 5s 40ms/step - loss: 0.7430 - accuracy: 0.8537 - val_loss: 0.4367 - val_accuracy: 0.8767
Epoch 2/20
36/36 [==============================] - 1s 29ms/step - loss: 0.3970 - accuracy: 0.8724 - val_loss: 0.3799 - val_accuracy: 0.8767
Epoch 3/20
36/36 [==============================] - 1s 28ms/step - loss: 0.3598 - accuracy: 0.8724 - val_loss: 0.3761 - val_accuracy: 0.8767
Epoch 4/20
36/36 [==============================] - 1s 28ms/step - loss: 0.3375 - accuracy: 0.8728 - val_loss: 0.3802 - val_accuracy: 0.8786
Epoch 5/20
36/36 [==============================] - 1s 28ms/step - loss: 0.3100 - accuracy: 0.8853 - val_loss: 0.3980 - val_accuracy: 0.8760
Epoch 6/20
36/36 [==============================] - 1s 28ms/step - loss: 0.2719 - accuracy: 0.8940 - val_loss: 0.4442 - val_accuracy: 0.8727
Epoch 7/20
36/36 [==============================] - 1s 33ms/step - loss: 0.2129 - accuracy: 0.9102 - val_loss: 0.5875 - val_accuracy: 0.8677
Epoch 8/20
36

In [ ]:
scores = model_02.evaluate(X_test,
y_test)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

245/245 [==============================] - 1s 2ms/step - loss: 0.3761 - accuracy: 0.8767
Loss: 0.376
Accuracy: 0.877


In [ ]:
scores = model.evaluate(X_test,
y_test)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

245/245 [==============================] - 1s 3ms/step - loss: 0.3761 - accuracy: 0.8791
Loss: 0.376
Accuracy: 0.879


In [ ]:
scores = model_01.evaluate(X_test,
y_test)
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

245/245 [==============================] - 1s 2ms/step - loss: 2.3850 - accuracy: 0.8158
Loss: 2.385
Accuracy: 0.816
